## Import Necessary Libraries

In [135]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras import models, layers
from keras_preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import  Adam
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [136]:
datagen = ImageDataGenerator()

In [137]:
train_path =  "./data/fruits-360_dataset/fruits-360/Training"
test_path = "./data/fruits-360_dataset/fruits-360/Test"

In [138]:
golden_1_train_path = "./data/fruits-360_dataset/fruits-360/Training/Apple Golden 1"

In [139]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2, 
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [140]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

Found 5240 images belonging to 6 classes.
Found 1751 images belonging to 6 classes.


In [141]:
train_class_names = list(train_generator.class_indices.keys())
train_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

In [142]:
test_class_names = list(test_generator.class_indices.keys())
test_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

## Begin Modeling

In [143]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

In [144]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)

In [145]:
print(train_img.shape)
print(test_img.shape)

(32, 120000)
(32, 120000)


In [146]:
train_y = np.reshape(train_labels[:,0], (32,1))
test_y = np.reshape(test_labels[:,0], (32,1))

In [147]:
np.random.seed(3)
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(20, activation='relu', input_shape=(120000,))) # 2 hidden layers
baseline_model.add(layers.Dense(7, activation='relu'))
baseline_model.add(layers.Dense(5, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [148]:
baseline_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = baseline_model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32)
#                     validation_data=(val_img, val_y))

Epoch 1/50
1/1 [==============================] - 0s 373ms/step - loss: 0.2971 - accuracy: 0.9062
Epoch 2/50
1/1 [==============================] - 0s 47ms/step - loss: 1.3970 - accuracy: 0.9062
Epoch 3/50


2022-12-05 15:36:33.734894: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 45ms/step - loss: 15.5816 - accuracy: 0.0938
Epoch 4/50
1/1 [==============================] - 0s 47ms/step - loss: 0.7373 - accuracy: 0.9062
Epoch 5/50
1/1 [==============================] - 0s 46ms/step - loss: 0.4615 - accuracy: 0.9062
Epoch 6/50
1/1 [==============================] - 0s 46ms/step - loss: 0.3244 - accuracy: 0.9062
Epoch 7/50
1/1 [==============================] - 0s 45ms/step - loss: 0.2823 - accuracy: 0.9062
Epoch 8/50
1/1 [==============================] - 0s 45ms/step - loss: 0.2737 - accuracy: 0.9062
Epoch 9/50
1/1 [==============================] - 0s 45ms/step - loss: 0.2708 - accuracy: 0.9062
Epoch 10/50
1/1 [==============================] - 0s 46ms/step - loss: 0.2684 - accuracy: 0.9062
Epoch 11/50
1/1 [==============================] - 0s 45ms/step - loss: 0.2659 - accuracy: 0.9062
Epoch 12/50
1/1 [==============================] - 0s 45ms/step - loss: 0.2634 - accuracy: 0.9062
Epoch 13/50
1/1 [====================

In [149]:
# sz = 200

# # Initializing the CNN
# model1 = Sequential()

# # First convolution layer and pooling
# model1.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))

# # Second convolution layer and pooling
# model1.add(Convolution2D(32, (3, 3), activation='relu'))
# # input_shape is going to be the pooled feature maps from the previous convolution layer
# model1.add(MaxPooling2D(pool_size=(2, 2)))

# # Flattening the layers
# model1.add(Flatten(input_shape = (28,28)))

# # Adding a fully connected layer
# model1.add(Dense(units=96, activation='relu'))
# model1.add(Dropout(0.40))
# model1.add(Dense(units=32, activation='relu'))
# model1.add(Dense(units=24, activation='softmax')) # softmax for more than 2

In [150]:
# model1.summary()

In [151]:
# model1.compile(optimizer='adam',
#                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#                metrics=['accuracy'])

In [152]:
# history = model1.fit(
#     train_generator,
#     validation_data = test_generator,
#     epochs = 5
# )